Here I tried to fine-tune a classification bert model on the sarc_emojis_1000 or the not_sarc_emojis_1000 dataset. The classification is for the 20 most common emojies in sarcastic comments. Here I used tensorflow instead of the transformers trainer. This is how I believe a classificatioin model should be trained, (one hot encoding and softmax layer) but it doesnt works.

In [ ]:
! gdown 1X1jZgrRaVH-h0xX3k8cSzT0kumIVlmwx #sarc_emojis_1000.csv
! gdown 1--LLxkODAryG3TgUHrcggfrYoXnr9UFe #not_sarc_emojis_1000.csv

Downloading...
From: https://drive.google.com/uc?id=1X1jZgrRaVH-h0xX3k8cSzT0kumIVlmwx
To: /content/sarc_emojis_1000.csv
100% 1.60M/1.60M [00:00<00:00, 177MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--LLxkODAryG3TgUHrcggfrYoXnr9UFe
To: /content/not_sarc_emojis_1000.csv
100% 1.68M/1.68M [00:00<00:00, 216MB/s]


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
df = pd.read_csv('not_sarc_emojis_1000.csv')
df.head()

,label,text
0,😒,… always searching for excuses.is so hard to a...
1,😒,still our transfers nonetheless. doesn't chang...
2,😒,waaaaaarom? waarom moeten mannen zeggen ‘ik zi...
3,😒,come on man!! you've got till november
4,😒,tu mere aadhe paune se jal mat when you can't...


Initializing empty arrays to store tokenized text.

In [ ]:
import numpy as np
from transformers import BertTokenizer

# set array dimensions
seq_len = 512
num_samples = len(df)

# initialize empty zero arrays
Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

# check shape
Xids.shape

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(df['text']):
    tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                   padding='max_length', add_special_tokens=True,
                                   return_tensors='tf')
    # assign tokenized outputs to respective rows in numpy arrays
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

One-hot encoding the sentiment

In [ ]:
emojies = ['😒','😂','👍','👌','🙄','😑','😊','😉','😏','😁','😔','😡','😐','😳','😜','😄','👏','☺','🤔','😃']
num_categories = 20

# first extract label column and convert to number
arr = [emojies.index(emoji) for emoji in df['label'].values]

# we then initialize the zero array
labels = np.zeros((num_samples, num_categories))

# set relevant index for each row to 1 (one-hot encode)
labels[np.arange(num_samples), arr] = 1

Build a tf.data.Dataset object using our input and label tensors. Then transform into the correct format for our model

In [ ]:
import tensorflow as tf

# create the dataset object
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
dataset = dataset.map(map_func)

Batch and shuffle our data

In [ ]:
# we will split into batches of 16
batch_size = 16

# shuffle and batch - dropping any remaining samples that don't cleanly
# fit into a batch of 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

90–10 split into training and validation data

In [ ]:

# set split size (90% training data) and calculate training set size
split = 0.9
size = int((Xids.shape[0]/batch_size)*split)

# get training and validation sets
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

Preparing the pre-trained model

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access pooled activations with [1]

# convert bert embeddings into 20 output classes
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(num_categories, activation='softmax', name='outputs')(x)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

#freeze bert layer
model.layers[2].trainable = False

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Fine-tuning the model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
  82/1125 [=>............................] - ETA: 7:18 - loss: 2.9251 - accuracy: 0.0846

KeyboardInterrupt: ignored

In [ ]:
from matplotlib import pyplot

#plotting results
def summarize_diagnostics(history):
  pyplot.subplot(211)
  pyplot.title('Cross Entropy Loss')
  pyplot.plot(history.history['loss'], color='blue', label='train')
  pyplot.plot(history.history['val_loss'], color='orange', label='test')
  pyplot.subplot(212)
  pyplot.title('Classification Accuracy')
  pyplot.plot(history.history['accuracy'], color='blue', label='train')
  pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
  pyplot.tight_layout()

In [ ]:
summarize_diagnostics(history)